1. Load Training data (numpy array of people)
    - x-values are stores to assign each person
    - y-values are needed to assign for each person
2. Read Video Stream using opencv
3. Extract Faces out of it
4. use knn to find the prediction of face (int)
5. map the predicted id to name of the user

In [1]:
import cv2
import numpy as np
import os

################### KNN ######################################

def distance(v1, v2):
    return np.sqrt(sum((v1-v2)**2))

def knn(train, test, k=5):
    dist = []
    
    for i in range(train.shape[0]):
        # get vector label
        ix = train[i,:-1]
        iy = train[i,-1]
        
        d = distance(test, ix)
        dist.append([d, iy])
        
    dk = sorted(dist)[:k]
    
    labels = np.array(dk)[:, -1]
    
    output = np.unique(labels, return_counts=True)
    
    index = np.argmax(output[1])
    return output[0][index]

######################################################################

cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip = 0
dataset_path = "./data/"

face_data = [] #Training data
labels = []

class_id = 0 #labels for given file
names = {} #mapping btw id and name

# Data preparation
for fx in os.listdir(dataset_path):
    if fx.endswith(".npy"):
        #create mapping between class_id and the name
        names[class_id] = fx[:-4]
        
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        # Create labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)
        
face_dataset = np.concatenate(face_data, axis=0)
face_lables = np.concatenate(labels, axis=0).reshape((-1, 1))

print(face_dataset.shape)
print(face_lables.shape)

trainset = np.concatenate((face_dataset,face_lables), axis=1)
print(trainset.shape)

while True:
    ref, frame = cap.read()
    if ref == False:
        cotinue
    
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    
    for face in faces:
        x,y,w,h = face
        
        # get face ROI
        offset = 10
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100,100))
        
        # predict label (out)
        out = knn(trainset, face_section.flatten())
        
        #display on the screen and rectangle around it
        pred_name = names[int(out)]
        cv2.putText(frame, pred_name,(x,y-10), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2, cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
    cv2.imshow("Faces", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

(248, 30000)
(248, 1)
(248, 30001)


ValueError: operands could not be broadcast together with shapes (67500,) (30000,) 